# Extended Phase Graphs Tutorial

Extended phase graphs (EPGs) are a powerful tool for MRI signal analysis.  Put simply, the EPG formalism (1) is a Fourier representation of transverse and longitudinal magnetization within a voxel and (2) EPG coefficients can very efficiently be propagated through common sequence operations (nutation, relaxation, gradients and diffusion) to easily and accurately simulate many common MRI pulse sequences.

## Setup
Our functions are in mrsigpy.py, so we will import that.

In [1]:
import numpy as np
import mrsigpy as mrs


## EPG Definition
The EPG basis consists of coefficients $F_n$ and $Z_n$, to represent transverse and longitudinal magnetization, respectively.  There are different conventions for these, but we use $F_n$ where $n$ is any integer and $Z_n$ where $n \ge 0$.  

The subscript n refers to the nubmer of cycles of dephasing, assumed to be an integer.  The transverse magnetization is expressed as $m_{xy} = m_x + i*m_y$, and longitudinal magnetization is $m_z$.  Then we write a real-valued vector $M = [m_{xy} m^\ast_{xy} m_z]$.

Now we consider a variable $z$ which is the position within the voxel, and then can express

$$m_{xy}(z) = \sum_{n=-\infty}^{\infty} F_n e^{2\pi i z}$$

This shows that $m_{xy}(z)$ is written using basis functions that are just twists ($e^{2\pi i z}$) and the basis coefficients are $F_n$.



## RF Rotations
The default for `epg_rf()` here is a simple $90_y$ degree excitation, which moves magnetization from $m_z$ to $m_x$, but the flip angle and phase (both in radians) can also be passed.

In [4]:
FZ = np.array([[0],[0],[1]])    # Equilibrium magnetization, along m_z
print(FZ)
FZ = mrs.epg_rf(FZ)             # 90 rotation along m_y
print("\nRotate from equilibrium about m_y to m_x:\n", FZ)
FZ = mrs.epg_rf(alpha=90.,phi=0.)
print("\nRotate from equilibrium about m_x to m_y:\n", FZ)
FZ = mrs.epg_rf(FZ,67.,90.)
print("\nRotating m_y around m_y does nothing:\n", FZ)

[[0]
 [0]
 [1]]

Rotate from equilibrium about m_y to m_x:
 [[1.000000e+00-6.123234e-17j]
 [1.000000e+00+6.123234e-17j]
 [6.123234e-17+0.000000e+00j]]

Rotate from equilibrium about m_x to m_y:
 [[0.000000e+00-1.j]
 [0.000000e+00+1.j]
 [6.123234e-17+0.j]]

Rotating m_y around m_y does nothing:
 [[ 1.90577075e-17-1.j]
 [ 1.90577075e-17+1.j]
 [-3.24392848e-17+0.j]]


## Gradients
Gradients simply move the $F_n$ coefficients around.  

In [5]:
FZ = mrs.epg_grad(FZ)      # FZ was excited by the RF
print(FZ)
FZ = mrs.epg_grad(FZ)
print(FZ)

[[ 0.00000000e+00-0.j  1.90577075e-17-1.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j]
 [-3.24392848e-17+0.j  0.00000000e+00+0.j]]
[[ 0.00000000e+00-0.j  0.00000000e+00-0.j  1.90577075e-17-1.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [-3.24392848e-17+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]]


In [6]:
FZ = np.array([[1],[1],[0]])
m = mrs.epg_FZ2spins(FZ,5)



ValueError: operands could not be broadcast together with shapes (6,) (3,) 